## ARIMA Setup

The purpose of this notebook is to read in the main catdef data from the pickle file, pre-process it, and stage it for processing by *auto.arima* in R.

### STEP 1.  Load Modules and Data

In [2]:
import pandas as pd
import numpy as np
import pickle
import math
import os

In [3]:
objects = []
with (open("all_data.pkl", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [4]:
print(str(len(objects)))
pdata=objects[0]

1


In [5]:
pdata.shape

(168, 20)

### STEP 2.  Extract Catdef for ARIMA

Having loaded the data, begin to stage it.  Before staging it, load the region mask file.  The region mask file indicates for each grid location, the region to which it belongs.  This information can be used to know which grid locations do *not* belong to any region so that no data for these irrelevant regions is written or so as to eliminate any chance of its analysis.

In [6]:
region_mask_file="Region_mask.csv"
region_data=list()
with open(region_mask_file,'r') as reader:
    temp_list=None
    for line in reader:
        pieces=line.split(',')
        temp_list=[int(p.strip()) for p in pieces]
        region_data.append(temp_list)

In [7]:
print(f"Region data has {len(region_data)} rows.")

Region data has 33 rows.


In [8]:
for row_idx in range(len(region_data)):
    row=region_data[row_idx]
    print(f"Row {row_idx} in region data has {len(row)} columns.")

Row 0 in region data has 37 columns.
Row 1 in region data has 37 columns.
Row 2 in region data has 37 columns.
Row 3 in region data has 37 columns.
Row 4 in region data has 37 columns.
Row 5 in region data has 37 columns.
Row 6 in region data has 37 columns.
Row 7 in region data has 37 columns.
Row 8 in region data has 37 columns.
Row 9 in region data has 37 columns.
Row 10 in region data has 37 columns.
Row 11 in region data has 37 columns.
Row 12 in region data has 37 columns.
Row 13 in region data has 37 columns.
Row 14 in region data has 37 columns.
Row 15 in region data has 37 columns.
Row 16 in region data has 37 columns.
Row 17 in region data has 37 columns.
Row 18 in region data has 37 columns.
Row 19 in region data has 37 columns.
Row 20 in region data has 37 columns.
Row 21 in region data has 37 columns.
Row 22 in region data has 37 columns.
Row 23 in region data has 37 columns.
Row 24 in region data has 37 columns.
Row 25 in region data has 37 columns.
Row 26 in region data 

In [3]:
extraction_dir="catdef_extraction_6mo"
if(not(os.path.exists(extraction_dir))):
    os.system("mkdir -v "+extraction_dir)

In [10]:
catdef_dims=pdata.apply(lambda row:row['catdef'].shape,axis=1).tolist()[0]


In [11]:
catdef_dims

(33, 37)

In [12]:
nan_counts_obs=set()
num_nan_dps=0
num_outside_ca_dps=0
num_data_written=0
for row_idx in range(catdef_dims[0]):
    print(f"row is {row_idx} of {catdef_dims[0]} ...")
    for col_idx in range(catdef_dims[1]):
        the_header=f"catdef_row.{row_idx}.col.{col_idx}"
        csv_target=f"{extraction_dir}/catdef.{the_header}.csv"
        the_data=pdata.apply(lambda row:row['catdef'].item((row_idx,col_idx)),axis=1).tolist()
        nan_data=[d for d in the_data if np.isnan(d)]
        num_nans=len(nan_data)
        if(num_nans>0):
            nan_counts_obs.add(num_nans)
            num_nan_dps=num_nan_dps+1
            #this means that any nan data is never written!
            continue
        region_id=region_data[row_idx][col_idx]
        if(region_id==0):
            num_outside_ca_dps=num_outside_ca_dps+1
            continue
        temp_df=pd.DataFrame.from_dict({the_header:the_data})
        temp_df.to_csv(csv_target,index=False)
        num_data_written=num_data_written+1
        #break
    #break
print(f"nan_counts_obs {nan_counts_obs}")
print(f"num nan data points : {num_nan_dps}")
print(f"Num non-CA data points {num_outside_ca_dps}")
print(f"Num data files written {num_data_written}")
#this shows that any series is either all-data or all-nan

row is 0 of 33 ...
row is 1 of 33 ...
row is 2 of 33 ...
row is 3 of 33 ...
row is 4 of 33 ...
row is 5 of 33 ...
row is 6 of 33 ...
row is 7 of 33 ...
row is 8 of 33 ...
row is 9 of 33 ...
row is 10 of 33 ...
row is 11 of 33 ...
row is 12 of 33 ...
row is 13 of 33 ...
row is 14 of 33 ...
row is 15 of 33 ...
row is 16 of 33 ...
row is 17 of 33 ...
row is 18 of 33 ...
row is 19 of 33 ...
row is 20 of 33 ...
row is 21 of 33 ...
row is 22 of 33 ...
row is 23 of 33 ...
row is 24 of 33 ...
row is 25 of 33 ...
row is 26 of 33 ...
row is 27 of 33 ...
row is 28 of 33 ...
row is 29 of 33 ...
row is 30 of 33 ...
row is 31 of 33 ...
row is 32 of 33 ...
nan_counts_obs {168}
num nan data points : 363
Num non-CA data points 418
Num data files written 440


### STEP 3.  Duplication for Multiple ARIMAs and Execution

After data is staged, it *must* be duplicated if ARIMA will be run with and without 'lambda="auto"'.

To duplicate it run the commands

```
mkdir -v catdef_extraction_6mo_lambda
cp -v catdef_extraction_6mo/* catdef_extraction_6mo_lambda
```

After staging/duplication the files, **inside the docker container built with the file Dockerfile.RDSNB**, run the Rscripts in a command like so : 

```
find catdef_extraction_6mo/*.csv|grep -Piv 'preds'|awk '{ print"./arima_train_6mo_cli.R " $0   }' |parallel -j 6 --eta
```

Change the "-j 6" for parallel to be perhaps "-j+0" to use all the cores or the 6 to some higher or lower value depending on the number of cores available.  The command can also be run inside a "screen" so as to help ensure that the program's execution is not interrupted.

Note that the arima_train_6mo_cli.R arima_train_6mo_lambda_cli.R scripts respectively are for analysis without and with 'lambda="auto"' setting used in the auto.arima call.  To run both analysis, simply use the command above, but substitute the desired script.  To help observe the differnce between the two scripts (namely the lambda=auto), simply run 

```
diff arima_train_6mo_cli.R arima_train_6mo_lambda_cli.R
```

.RData files are generated by the script which contain the arima models and predictions for each of the 24 test periods as well as a prediction for the "25th" test period (Jan 2017).  These can be loaded in R to examine the coefficients and other specifics of the individual models.